## CNN Attempt for ISIC 2024 - Skin Cancer Detection with 3D-TBP

Present issue is the massive imbalance in the dataset of positives compared to negatives. To address this issue, I will utilize a custom sampler object called `ImbalancedDatasetSampler` when initiating DataLoader objects.

Another step to increase the effectiveness of CNN on binary classification of skin cancer on the image dataset is using an ensemble of CNN's which is commonly known to increase overall performances.

## Import Libraries

In [26]:
"""
Import libraries
"""
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torcheval.metrics.functional import binary_auroc
from torchvision import models


from io import BytesIO
import h5py
import io
import random
import os
from PIL import Image
import cv2
from tqdm import tqdm
import copy
import gc
import time
from collections import defaultdict

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# cuda gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Training Configuration

In [19]:
CONFIG = {
    "seed": 42,
    "epochs": 20,
    "img_size": 336,
    "train_batch_size": 150,
    "valid_batch_size": 200,
    "learning_rate": 1e-5,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 1e-6,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": device,

    }

In [20]:
"""
Set seed and 
"""


SEED = 111
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

ROOT_DIR = "../data"
TRAIN_CSV = f"{ROOT_DIR}/train-metadata.csv"
TRAIN_HDF = f"{ROOT_DIR}/train-image.hdf5"
TEST_CSV = f'{ROOT_DIR}/test-metadata.csv'
TEST_HDF = f'{ROOT_DIR}/test-image.hdf5'
SAMPLE = f'{ROOT_DIR}/sample_submission.csv'

## Reading in the Data

Code to read in data is from Eren's notebook: https://www.kaggle.com/code/metlnfoor/resnet34-removing-hair-under-sampling/notebook

In [21]:
train_metadata = pd.read_csv(TRAIN_CSV, low_memory=False)
test_metadata = pd.read_csv(TEST_CSV)

In [23]:
def read_images_from_hdf5(file_path):
    images = {}
    try:
        with h5py.File(file_path, 'r') as file:
            for key in tqdm(file.keys(), desc="Reading Files"):
                try:
                    image_data = file[key][()]
                    image = Image.open(io.BytesIO(image_data))
                    images[key] = image
                except Exception as e:
                    print(f"Error! from {key}: {e}")
    except Exception as e:
        print(f"Error occured while reading files : {e}")
    
    return images

In [25]:
train_images = read_images_from_hdf5(TRAIN_HDF)
test_images = read_images_from_hdf5(TEST_HDF)

Reading Files: 100%|██████████| 3/3 [00:00<00:00, 1504.41it/s]


## Remove Hair function and Skin Tone standardization

In [27]:
# This function from https://www.kaggle.com/competitions/isic-2024-challenge/discussion/519735
def remove_hair(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
        blackhat = cv2.morphologyEx(gray,cv2.MORPH_BLACKHAT,kernel)

        _, thresh = cv2.threshold(blackhat, 10 ,255, cv2.THRESH_BINARY)
        inpainted_image = cv2.inpaint(image, thresh, 1, cv2.INPAINT_TELEA)
        return inpainted_image

In [ ]:
def tone_stand(image):
    

    return final_img

## Dataset Class

In [7]:
class ISIC_2024(Dataset):
    def __init__(self,pil_images,metadata,transform=None,test=False):
        self.pil_images = pil_images
        self.metadata = metadata
        self.transform = transform
        self.test= test
        
    def __len__(self):
        return len(self.metadata)
    
    # This function from https://www.kaggle.com/competitions/isic-2024-challenge/discussion/519735
    def remove_hair(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
        blackhat = cv2.morphologyEx(gray,cv2.MORPH_BLACKHAT,kernel)

        _, thresh = cv2.threshold(blackhat, 10 ,255, cv2.THRESH_BINARY)
        inpainted_image = cv2.inpaint(image, thresh, 1, cv2.INPAINT_TELEA)
        return inpainted_image
    
    def __getitem__(self,idx):
        isic_id = self.metadata.iloc[idx,0]
        cleaned_image = remove_hair(np.array(self.pil_images[isic_id]))
        image = Image.fromarray(cleaned_image)
        if self.transform:
            image = self.transform(image)
        if self.test:
            return image
        label = self.metadata.iloc[idx,-1]
        return image,label

## Get DataLoaders

In [8]:
"""
Define DataLoader Transforms
"""

data_transforms = {
    "train": A.Compose([
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.4815, 0.4578, 0.4082], 
                std=[0.2686, 0.2613, 0.2758], 
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()
    ], p=1.),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.4815, 0.4578, 0.4082], 
                std=[0.2686, 0.2613, 0.2758], 
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2(),
        ], p=1.)
}

In [9]:
def prepare_loaders(df, df_pseudo):
    
    train_dataset = ISICDataset_for_Train(df, TRAIN_HDF, df_pseudo, TEST_HDF, transforms=data_transforms["train"])
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    return train_loader

## Define Model 1: ResNet152

In [10]:
model1 = models.resnet152(weights = models.ResNet152_Weights.IMAGENET1K_V1)
model1.fc = nn.Linear(model1.fc.in_features, 2) #change to 200 outputs for the 200 classes
model1 = model1.to(device)

## Define Criterion and Optimizer

In [11]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model1.parameters(), lr=CONFIG['learning_rate'], 
                       weight_decay=CONFIG['weight_decay'])


## Get Scheduler

In [12]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

scheduler = fetch_scheduler(optimizer)

## Training and Val Loop Function

In [13]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
            
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        auroc = binary_auroc(input=outputs.squeeze(), target=targets).item()
        
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss, epoch_auroc


def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for _, data in bar:        
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)

        outputs = model(images).squeeze()
        loss = criterion(outputs, targets)

        auroc = binary_auroc(input=outputs.squeeze(), target=targets).item()
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, Valid_Auroc=epoch_auroc,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss, epoch_auroc
    

## Define inference function

In [14]:
def inference(df_test, file_test_hdf, model):
    model.eval();
    test_dataset = ISICDataset(df_test, file_test_hdf, transforms=data_transforms["valid"])
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['valid_batch_size'],
                               shuffle=False, pin_memory=True)
    
    preds = []
    with torch.no_grad():
        bar = tqdm(enumerate(test_loader), total=len(test_loader))
        for _, data in bar:        
            images = data['image'].to(CONFIG["device"], dtype=torch.float)        
            outputs = model(images)
            preds.append( outputs.detach().cpu().numpy() )
    preds = np.concatenate(preds).flatten()
    return preds

## Training Loop

In [15]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        # Inference test data
        df_test = pd.read_csv(TEST_CSV)
        df_test["target"] = 0 # dummy
        df_test["target"] = inference(df_test, TEST_HDF, model)
        
        # create training dataset
        train_loader = prepare_loaders(df, df_test, fold=CONFIG["fold"])
        del df_test
        gc.collect()
        
        # train (1epoch)
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        del train_loader
        gc.collect()
        
        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                        epoch=epoch)
        del valid_loader
        gc.collect()
        
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid AUROC'].append(val_epoch_auroc)
        history['lr'].append( scheduler.get_lr()[0] )
        
        best_model_wts = copy.deepcopy(model.state_dict())
        PATH = "model_epoch{:.0f}.bin".format(epoch)
        torch.save(model.state_dict(), PATH)
        
        # deep copy the model
        if best_epoch_auroc <= val_epoch_auroc:
           print(f"Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
           best_epoch_auroc = val_epoch_auroc
           best_model_wts = copy.deepcopy(model.state_dict())
           PATH = "AUROC{:.4f}_Loss{:.4f}_epoch{:.0f}.bin".format(val_epoch_auroc, val_epoch_loss, epoch)
           torch.save(model.state_dict(), PATH)
           # Save a model file from the current directory
           print(f"Current model Saved")
            
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [16]:
model1, history = run_training(model1, optimizer, scheduler,
                              device=device,
                              num_epochs=CONFIG['epochs'])

[INFO] Using GPU: NVIDIA GeForce RTX 4060 Ti



100%|██████████| 1/1 [00:00<00:00,  2.23it/s]


ValueError: Length of values (6) does not match length of index (3)